# 1. Instalar e Importar Librerías Necesarias

En esta sección instalaremos e importaremos todas las librerías necesarias para el procesamiento de PDFs, vectorización, tokenización y uso de modelos ligeros multilenguaje compatibles con Ollama.

In [ ]:
# Instalar librerías necesarias
%pip install PyPDF2 sentence-transformers langchain faiss-cpu pdf2image pytesseract pillow opencv-python tiktoken chromadb langchain-community

# Importar librerías
import os
import subprocess
from pdf2image import convert_from_path
import pytesseract
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle
import cv2
from PIL import Image
from pathlib import Path
import tiktoken

# 2. Configuración y seteo de binarios

En esta sección construiremos todas las configuraciones necesarias, path a binario y demás

In [ ]:

BASE_DIR = Path().resolve()
PDF_FOLDER = os.path.join(BASE_DIR, "pdfs")
print(f"PDF_FOLDER: {PDF_FOLDER}")

INDEX_PATH = os.path.join(BASE_DIR, "faiss_index.bin")
CHUNKS_PATH = os.path.join(BASE_DIR, "chunks.pkl")
EMBEDDINGS_PATH = os.path.join(BASE_DIR, "embeddings.npy")
OUTPUT_FOLDER = os.path.join(BASE_DIR, "ocr_textos")

try:
    import chromadb
    from chromadb.config import Settings
except Exception as e:
    print("Chromadb no instalado. Ejecuta: pip install chromadb")
    raise

#tesseract
tesseract_cmd = os.path.join(os.getcwd(), "tesseract", "tesseract.exe")
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

#poppler
poppler_path = os.path.join(os.getcwd(), "poppler", "bin")

required_bins = ["pdfinfo.exe", "pdftoppm.exe"]
for b in required_bins:
    if not os.path.exists(os.path.join(poppler_path, b)):
        raise FileNotFoundError(f"No se encontró {b} en {poppler_path}")

# 3. Cargar modelo multilenguaje

En esta sección se carga un modelo para realizar los embeddings. Debe ser el mismo con el cual se lee después

In [ ]:
model_name = "paraphrase-multilingual-mpnet-base-v2"
embedder = SentenceTransformer(model_name)

# 4. Función para mejorar el contraste de los pdf

En esta sección se mejora el pdf, realzando el contraste para mejorar la identificación del texto

In [ ]:
def preprocess_image(pil_image):
    cv_image = np.array(pil_image)
    cv_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2GRAY)
    # Aplicar umbral para mejorar contraste
    _, thresh = cv2.threshold(cv_image, 180, 255, cv2.THRESH_BINARY)
    return Image.fromarray(thresh)

# 5. Función OCR de imágenes dentro del pdf

En esta sección se revisan si hay imagenes en pdf para extraer texto

In [ ]:
def extract_text_from_image_pdf(pdf_file, psm=6, chunk_size=200, overlap=50, context_window=1):
    pages = convert_from_path(pdf_file, dpi=300, poppler_path=poppler_path)
    all_chunks = []

    pdf_name = os.path.splitext(os.path.basename(pdf_file))[0]
    txt_output = os.path.join(OUTPUT_FOLDER, f"{pdf_name}ocr.txt")

    with open(txt_output, "w", encoding="utf-8") as f:
        f.write(f"========== OCR de {os.path.basename(pdf_file)} ==========\n")

        for i, page in enumerate(pages):
            processed_page = preprocess_image(page)
            text = pytesseract.image_to_string(processed_page, lang='spa', config=f'--psm {psm}').strip()

            f.write(f"\n--- Página {i + 1} ---\n")
            if text:
                f.write(text + "\n")
                words = text.split()

                # Si no hay palabras, saltar
                if len(words) == 0:
                    f.write("[Sin texto detectado]\n")
                    print(f"⚠️ {os.path.basename(pdf_file)} - Página {i + 1}: sin texto detectado")
                    continue

                # Generar chunks con solapamiento (sliding window)
                page_chunks = []
                step = max(1, chunk_size - overlap)
                for start in range(0, len(words), step):
                    chunk_words = words[start:start + chunk_size]
                    chunk_text = ' '.join(chunk_words)
                    page_chunks.append({
                        'text': chunk_text,
                        'page': i + 1,
                        'start': start,
                        'end': start + len(chunk_words)
                    })

                # Construir contexto para cada chunk (concatenar neighbors según context_window)
                for idx, c in enumerate(page_chunks):
                    parts = []
                    for k in range(idx - context_window, idx + context_window + 1):
                        if 0 <= k < len(page_chunks):
                            parts.append(page_chunks[k]['text'])
                    c['text_with_context'] = ' '.join(parts)
                    c['chunk_id'] = f"{pdf_name}_p{i + 1}_c{idx}"

                all_chunks.extend(page_chunks)
                print(f"📄 {os.path.basename(pdf_file)} - Página {i + 1}: {len(words)} palabras extraídas -> {len(page_chunks)} chunks")
            else:
                f.write("[Sin texto detectado]\n")
                print(f"⚠️ {os.path.basename(pdf_file)} - Página {i + 1}: sin texto detectado")

    print(f"📝 Texto OCR guardado en: {txt_output}")
    return all_chunks


# 6. Procesamos todos los pdfs

En esta sección se recolentan los pdfs y se procesan

In [ ]:
pdf_files = [os.path.join(PDF_FOLDER, f) for f in os.listdir(PDF_FOLDER) if f.lower().endswith('.pdf')]

all_chunks = []
for pdf_file in pdf_files:
    chunks = extract_text_from_image_pdf(pdf_file)
    print(f"{os.path.basename(pdf_file)} -> {len(chunks)} chunks generados\n")
    all_chunks.extend(chunks)

print(f"📦 Total de chunks generados: {len(all_chunks)}")
print(f"📁 Archivos OCR guardados en: {OUTPUT_FOLDER}")

# 7. Generación de embeddings

En esta sección se generan los embeddings

In [ ]:
if len(all_chunks) == 0:
    raise ValueError("No se generaron chunks. Revisa los PDFs y el OCR.")

# Soportar tanto lista de strings (antiguo) como lista de dicts con metadatos
if isinstance(all_chunks[0], dict):
    texts_to_embed = [c.get('text_with_context', c.get('text')) for c in all_chunks]
else:
    texts_to_embed = all_chunks

embeddings = embedder.encode(texts_to_embed, show_progress_bar=True, convert_to_numpy=True)
print(f"Se generaron {embeddings.shape[0]} vectores.")


# 8. Creamos el indice FAISS

En esta sección se crea el indice FAISS (basicamente es para manipular mejor los vectores, obtenerlos eficientemente bla bla)

In [ ]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print(f"Índice FAISS creado con {index.ntotal} vectores.")

# 9. Guardamos indices y datos

En esta sección se guardan los indices

In [ ]:
# Tokenize chunks (try tiktoken, fallback to word-splitting)
try:
    
    # Try to get an encoder for a common model; if not available, use cl100k_base
    try:
        encoder = tiktoken.encoding_for_model("gpt-4o-mini")
    except Exception:
        encoder = tiktoken.get_encoding("cl100k_base")

    for c in all_chunks:
        text = c.get('text_with_context', c.get('text', ''))
        try:
            token_ids = encoder.encode(text)
            c['tokens'] = token_ids
            c['token_count'] = len(token_ids)
        except Exception:
            words = text.split()
            c['tokens'] = words
            c['token_count'] = len(words)
except Exception:
    # Fallback simple tokenization by words
    for c in all_chunks:
        text = c.get('text_with_context', c.get('text', ''))
        tokens = text.split()
        c['tokens'] = tokens
        c['token_count'] = len(tokens)

# Guardar índice y datos
faiss.write_index(index, INDEX_PATH)
np.save(EMBEDDINGS_PATH, embeddings)
with open(CHUNKS_PATH, "wb") as f:
    pickle.dump(all_chunks, f)

# Resumen rápido
total_tokens = sum(c.get('token_count', 0) for c in all_chunks)
max_chunk = max(all_chunks, key=lambda x: x.get('token_count', 0)) if all_chunks else None
print("FAISS index, embeddings y chunks guardados correctamente.")
print(f"Chunks guardados: {len(all_chunks)} | Total tokens (aprox): {total_tokens}")
if max_chunk:
    print(f"Chunk con más tokens: {max_chunk.get('chunk_id', '<sin id>')} -> {max_chunk.get('token_count')} tokens")


# 10. Revisamos cuantos chunks se crearon

En esta sección revisamos cuantos chunks se crearon

In [ ]:
with open(CHUNKS_PATH, "rb") as f:
    loaded_chunks = pickle.load(f)
print(f"Número de chunks cargados: {len(loaded_chunks)}")
first = loaded_chunks[0]
if isinstance(first, dict):
    print("Ejemplo de chunk id:", first.get('chunk_id'))
    print("Página:", first.get('page'))
    print("Texto (primeros 200 chars):", first.get('text')[:200], "...")
    print("Texto con contexto (primeros 200 chars):", first.get('text_with_context')[:200], "...")
else:
    print("Ejemplo de chunk:", loaded_chunks[0][:200], "...")


# Indexación y motores de búsqueda semántica

En esta sección añadimos información y ejemplos prácticos para:  
- Indexar tus embeddings en una base de vectores (Chroma/Pinecone/Qdrant/FAISS local) y poder hacer búsquedas por similitud.  
- Conectar un motor tipo chatbot usando un framework (por ejemplo LangChain) que coordine la recuperación + generación de respuestas.

Conceptos clave:

- Qué almacenar: el embedding (vector), el texto original o `text_with_context`, y metadatos (documento, página, offsets, chunk_id).  
- Trade-offs: bases locales (FAISS/Chroma local) son simples y baratas; servicios gestionados (Pinecone, Qdrant Cloud) escalan mejor y ofrecen features (replicación, métricas, control de versiones).  
- Estrategia común: índice "coarse" (resúmenes/páginas) + índice "fine" (chunks con contexto). Recupera coarse -> restringe search en fine -> ensamblar ordenado -> enviar al LLM.


In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

MODEL_NAME = "paraphrase-multilingual-mpnet-base-v2"
PERSIST_DIR = "./chroma_db"

embedder = SentenceTransformer(MODEL_NAME)

client = chromadb.PersistentClient(path=PERSIST_DIR)

collection_name = "inflecta_chunks"

try:
    collection = client.get_collection(collection_name)
except Exception:
    collection = client.create_collection(collection_name)

texts = [c.get('text_with_context', c.get('text', '')) for c in all_chunks]
ids = [c.get('chunk_id', f"chunk_{i}") for i, c in enumerate(all_chunks)]
metadatas = [{k: v for k, v in c.items() if k in ('chunk_id', 'page', 'start', 'end')} for c in all_chunks]

embeddings = embedder.encode(texts, convert_to_numpy=True).tolist()

collection.add(ids=ids, documents=texts, metadatas=metadatas, embeddings=embeddings)

print(f"Añadidos {len(ids)} vectores a Chroma en {PERSIST_DIR} (colección: {collection_name})")


In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MODEL_NAME = "paraphrase-multilingual-mpnet-base-v2"
PERSIST_DIR = "./chroma_db"

if not OPENAI_API_KEY:
    raise ValueError("⚠️ Falta OPENAI_API_KEY en tu archivo .env")

embeddings = SentenceTransformerEmbeddings(model_name=MODEL_NAME)

vectordb = Chroma(
    persist_directory=PERSIST_DIR,
    collection_name="inflecta_chunks",
    embedding_function=embeddings
)

retriever = vectordb.as_retriever(search_kwargs={"k": 4})

llm = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.0)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    chain_type="stuff"
)

print("💬 Chat RAG iniciado (escribe 'salir' para terminar)\n")

while True:
    pregunta = input("Tú: ")
    if pregunta.lower() in ["salir", "exit", "quit"]:
        print("👋 Fin del chat.")
        break

    respuesta = qa_chain({"question": pregunta})
    print("🤖:", respuesta["answer"])
